# SARL for Portfolio Management on DJ30
This tutorial is to demonstrate an example of using SARL to do portfolio management on DJ30

## Step1: Import Packages

In [2]:
import warnings

warnings.filterwarnings("ignore")
import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.datasets.builder import build_dataset
from trademaster.trainers.builder import build_trainer

## Step2: Import Configs

In [3]:

parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_dj30_sarl_sarl_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
args, _ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

In [4]:
cfg

Config (path: E:\workspace\RA\TradeMaster\configs\portfolio_management\portfolio_management_dj30_sarl_sarl_adam_mse.py): {'data': {'type': 'PortfolioManagementDataset', 'data_path': 'data/portfolio_management/dj30', 'train_path': 'data/portfolio_management/dj30/train.csv', 'valid_path': 'data/portfolio_management/dj30/valid.csv', 'test_path': 'data/portfolio_management/dj30/test.csv', 'tech_indicator_list': ['high', 'low', 'open', 'close', 'adjcp', 'zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5', 'zd_10', 'zd_15', 'zd_20', 'zd_25', 'zd_30'], 'length_day': 5, 'initial_amount': 10000, 'transaction_cost_pct': 0.001, 'test_dynamic_path': 'data/portfolio_management/dj30/DJI_label_by_DJIindex_3_24_-0.25_0.25.csv'}, 'environment': {'type': 'PortfolioManagementSARLEnvironment'}, 'trainer': {'type': 'PortfolioManagementSARLTrainer', 'agent_name': 'ddpg', 'if_remove': False, 'configs': {'framework': 'tf2'}, 'work_dir': 'work_dir/portfolio_management_dj30_sarl_sarl_adam_mse'}, 'loss': {'type

## Step3: Build Dataset

In [5]:
dataset = build_dataset(cfg)

## Step4: Build Trainer

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
work_dir = os.path.join(ROOT, cfg.trainer.work_dir)

if not os.path.exists(work_dir):
    os.makedirs(work_dir)
cfg.dump(osp.join(work_dir, osp.basename(args.config)))

trainer = build_trainer(cfg, default_args=dict(dataset=dataset, device = device))

| Arguments Keep work_dir: E:\workspace\RA\TradeMaster\work_dir/portfolio_management_dj30_sarl_sarl_adam_mse


## Step5: Train, Valid and Test

In [ ]:
trainer.train_and_valid()

2023-02-28 00:57:53,942	INFO trainer.py:2321 -- Executing eagerly (framework='tf2'), with eager_tracing=False. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
2023-02-28 00:57:53,943	WARNING deprecation.py:46 -- DeprecationWarning: `config['prioritized_replay']` has been deprecated. Replay prioritization specified at new location config['replay_buffer_config']['prioritized_replay'] will be overwritten. This will raise an error in the future!
2023-02-28 00:57:53,943	WARNING deprecation.py:46 -- DeprecationWarning: `config[prioritized_replay_alpha]` has been deprecated. config['replay_buffer_config'][prioritized_replay_alpha] should be used for Q-Learning algorithms. Ignore this warning if you are not using a Q-Learning algorithm and still provide prioritized_replay_alpha. This will raise an error in the future!
2023-02-28 00:57:53,944	WARNING 

Train Episode: [1/20]


In [ ]:
trainer.test()